# TouchTerrain standalone in a jupyter notebook
Chris Harding, Sep 7, 2020 (<charding@iastate.edu>)

- this jupyter notebook runs a standalone version of TouchTerrain, similar to `TouchTerrain_standalone.py`
- this notebook needs to be run in Python __3.x__  (I'm using 3.7)
- Use pip to build and install a package called touchterrain (all lowercase!), which contains all functions needed to run in standalone and in server mode. This will also install all dependencies needed(!)


- you can run pip either in a terminal or inside a jupyer cell (next cell)
- to run it in a separate terminal, `cd` to the folder that contains the `setup.py` file (which will contain a folder called touchterrain)
    - enter:  `pip install .` 
    - note the . at the end, which will make pip run the script in setup.py
    - this will install all dependencies (including earthengine-api) 
- on Windows, if pip gives you trouble with __gdal__, get the whl file from [here](https://www.lfd.uci.edu/~gohlke/pythonlibs/) and install it separatly with `pip install <whl file>` then go back and try the full install again
- on Mac, gdal should be avaiable via [conda-forge](https://anaconda.org/conda-forge/gdal), however I have run into trouble with shared library mismatches which I could not resolve! 




In [ ]:
# if you have already installed touchterrain, skip this cell!

# Run this cell (Shift-Enter) to have pip install the touchterrain module and all its dependencies
# Warning: this can take a awhile! You may not see any message during the install but you should see the full log when pip is done (or when you got errors)
!pip3 install .

## If you're not going to use Earth Engine's (EE) online DEM data skip the next section and go to _Running Touchterrain standalone_


### Preparing to use Google Earth Engine online DEM rasters 

- You don't need this if you only want to import terrain from locally stored raster files (geotiffs)!
- TouchTerrain can use DEM data from Google Earth Engine (given the corners of the area), but you need to first request a developer account and set up an authentication file
- (This dev account is different from your standard Google (Gmail, etc.) account!)
- Getting the account is free and entitles you to a modest number of requests (4 per seconds), which are also free. To request got to https://signup.earthengine.google.com/, you'll get and email with a file. 
- refer to the part __Setting Up Authentication Credentials__ https://developers.google.com/earth-engine/python_install (ignore the stuff above it, as you should already have installed all the needed packages when you installed the earthengine-api package ...)


- after ee has been imported it will need to be initialized (`ee.Initialize()`) at which point it will run an autentication. The autehntication will try to access a local authentication file, which is created by running `ee.Authenticate()` once. 
- To run `ee.Authenticate()`, comment in and run the next cell, which will end up creating a authentication file for your local system. After you have done this ONCE, you should not have to use `ee.Authenticate()`, so you can comment it out again.


In [ ]:
#ee.Authenticate()

- When you use: `ee.Authenticate()` this text should appear and you should be redirected to a webpage:
```
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does  not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=5172225062...

```
- the web page will have you select a Google account for use with ee and give you a code
- paste in the code: `Enter verification code: <your code>` and hit Enter, you should get `Successfully saved authorization token.`
- this will create a folder `.config` in your home folder and create an authentication file (in a earthengine folder) that `ee.Initialize()` will to authenticate you. 
- `import TouchTerrainEarthEngine` (below) will internally run `ee.Initialize()`. You should see: `EE init() worked with .config/earthengine/credentials`  
- Once this is done (and the authentication worked) you can access the online DEM layers.

In [1]:
# import packages
import os, sys
from pprint import pprint

# earthengine-api and touchterrain should have been installed via pip earlier
import ee 
from touchterrain.common import TouchTerrainEarthEngine as TouchTerrain

INFO:root:EE init() worked with .config/earthengine/credentials


## Running Touchterrain standalone (local and online DEM data)

- Put your settings into the dictionary below and hit Shift-Enter


- for more info on the settings, look at the ReadMe on https://github.com/ChHarding/TouchTerrain_for_CAGEO
- note, however, that the settings given below are in Python syntax, whereas the ReadMe describes the JSON syntax used in the config file
- both are very similar except for None and True/False

``` 
    Python:  JSON:
    None     null
    True     true
    False    false
```



In [9]:
args = {
    # DEM/Area to print
    
    # A: use local DEM raster (geotiff)
    #"importedDEM": "stuff/pyramid.tif",  # path to the geotif in relation to where this notebook sits
    
    # B: use area and a DEM online source via EarthEngine
    "importedDEM": None,
    "DEM_name": "USGS/NED",   # DEM source
    # the following defines the area, but you can also define it by hand (see digitizing) 
    "bllat": 44.50185267072875,   # bottom left corner lat
    "bllon": -108.25427910156247, # bottom left corner long
    "trlat": 44.69741706507476,   # top right corner lat
    "trlon": -107.97962089843747, # top right corner long

    # 3D print parameters
    "tilewidth": 200,  # width of each tile in mm, (tile height will be auto calculated)
    "printres": 0.4,  # resolution (horizontal) of 3D printer in mm, should be your NOZZLE size or just a bit less! 
                      # Using something like 0.01 will NOT print out a super detailed version as you slicer will remove
                      # super fine details anyway! You'll just wait a long time and get a super large STL file!
    
                      # If you want the equivalent of the original resolution of the DEM, use -1 (But again, think of the slicer ...)
                      # This will work for any local DEM but only for small online DEMs b/c 
                      # Google now (Mar. 2020) caps raster downloads at 10 Mega pixels!
    
    "ntilesx": 1, # number of tiles in x  
    "ntilesy": 1, # number of tiles in y    

    "basethick": 0.5,   # thickness (in mm) of printed base
    "zscale": 1,      # elevation (vertical) scaling
    "fileformat": "STLb",  # format of 3D model files: "obj" wavefront obj (ascii),
                           #   "STLa" ascii STL or "STLb" binary STL.
                           #   To export just the (untiled) raster (no mesh), use "GeoTiff" 
    "zip_file_name": "myterrain",   # base name of zipfile, .zip will be added

    
    # Expert settings
    "tile_centered": False, # True-> all tiles are centered around 0/0, False, all tiles "fit together"
    "CPU_cores_to_use" : 0, # 0: use all available cores, None: don't use multiprocessing (single core only)
                            # multi-core will be much faster for more than 1 tile 
    "max_cells_for_memory_only" : 5000^2, # if number of raster cells is bigger than this, use temp_files instead of memory.
                            # set this very high to force use of memory and lower it if you run out of memory
    "no_bottom": False,   # omit bottom triangles? Most slicers still work and it makes smaller files
    "no_normal": True,    # Don't calculate normals for triangles. This is significantly faster but some viewer may need them.
    "bottom_image": None, # 1 band greyscale image used for bottom relief
    "ignore_leq": None,   # set all values <= this to NaN so they don't print
    "lower_leq": None,    # e.g. [0.0, 2.0] values <= 0.0 will be lowered by 2mm in the final model
    "unprojected": False, # don't project to UTM (for EE rasters only)
    "projection": None,   # None means use the closest UTM zone. Can be a EPSG number (int!) instead but not all work. 
    "only" : None,        # if not None: list with x and y tile index (1 based) of the only tile to process
                          #   e.g. [1,1] will only process the tile in upper left corner, [2,1] the tile right to it, etc.
    "importedGPX": [],    # list of gpx path file(s) to be use (optional: see next cell)
}
########################################################

# if we want to work on a local raster, get the full pathname to it
if args["importedDEM"] != None: 
    from os.path import abspath
    args["importedDEM"]= abspath(args["importedDEM"]) 
    print("reading in local DEM:", args["importedDEM"])
print("settings stored, ready to process")

settings stored, ready to process


### Optional: Drape GPX path file(s) over the terrain (thanks to KohlhardtC!)
- you can drape one or more gpx (path) files over your terrain
- the gpx file(s) have to be part of list (`importedGPX`)
- `gpxPathHeight` (meters) defines how much a path is elevated above the terrain. Use a negative number to create a trench.
- `gpxPixelsBetweenPoints` (meters) lets you reduce the number of point in your path to place a point only every X meters. This can help to simplify complex patterns.
- `gpxPathThickness` (meters) controls the thickness of the path


In [3]:
# Note: you must uncomment the last line in this cell to actually use these gpx settings!
from touchterrain.common.TouchTerrainGPX import *
gpx_args = {   
    # Area for using the example GPX test files
    "bllat": 39.32205105794382,   # bottom left corner lat
    "bllon": -120.37497608519418, # bottom left corner long
    "trlat": 39.45763749030933,   # top right corner lat
    "trlon": -120.2002248034559, # top right corner long

    "importedGPX": # gpx example files.
                 ["stuff/gpx-test/DLRTnML.gpx",
                  "stuff/gpx-test/DonnerToFrog.gpx",
                  "stuff/gpx-test/CinTwistToFrog.gpx",
                  "stuff/gpx-test/sagehen.gpx",
                  "stuff/gpx-test/dd-to-prosser.gpx",
                  "stuff/gpx-test/alder-creek-to-crabtree-canyon.gpx",
                  "stuff/gpx-test/ugly-pop-without-solvang.gpx",
                  "stuff/gpx-test/tomstrail.gpx"   
                 ],
     "gpxPathHeight": 10,  # Currently we plot the GPX path by simply adjusting the
                           # raster elevation at the specified lat/lon,
                           # therefore this is in meters. Negative numbers are ok 
                           # and put a dent in the mdoel  
     "gpxPixelsBetweenPoints" : 20, # GPX Files haves a lot of points. A higher 
                                    #number will create more space between lines drawn
                                    # on the model and can have the effect of making the paths look a bit cleaner 
     "gpxPathThickness" : 5, # Stack paralell lines on either side of primary line 
                             # to create thickness. 
                             # A setting of 1 probably looks the best
}

# uncomment the next line if you want to use gpx_args!
#args = {**args, **gpx_args}; print(args) # merge gpx_args into args, ** unrolls dicts

{'importedDEM': None, 'DEM_name': 'USGS/NED', 'bllat': 39.32205105794382, 'bllon': -120.37497608519418, 'trlat': 39.45763749030933, 'trlon': -120.2002248034559, 'tilewidth': 200, 'printres': 0.4, 'ntilesx': 1, 'ntilesy': 1, 'basethick': 0.5, 'zscale': 1, 'fileformat': 'STLb', 'zip_file_name': 'myterrain', 'tile_centered': False, 'CPU_cores_to_use': 0, 'max_cells_for_memory_only': 5002, 'no_bottom': False, 'no_normal': True, 'bottom_image': None, 'ignore_leq': None, 'lower_leq': None, 'unprojected': False, 'projection': None, 'only': None, 'importedGPX': ['stuff/gpx-test/DLRTnML.gpx', 'stuff/gpx-test/DonnerToFrog.gpx', 'stuff/gpx-test/CinTwistToFrog.gpx', 'stuff/gpx-test/sagehen.gpx', 'stuff/gpx-test/dd-to-prosser.gpx', 'stuff/gpx-test/alder-creek-to-crabtree-canyon.gpx', 'stuff/gpx-test/ugly-pop-without-solvang.gpx', 'stuff/gpx-test/tomstrail.gpx'], 'gpxPathHeight': 10, 'gpxPixelsBetweenPoints': 20, 'gpxPathThickness': 5}


### Optional: Show map and digitize print area (box, circle or polygon)
- the next 2 cells will show you a map and will let you manually define a shape, either a rectangle (box), a circle or a polygon
- it will show you a red box of the area defined earlier (with bllat, etc.) in case you want to digitize inside it. However, you're free to digitize anywhere on the map! Your digitized shape will always override the red box.
- Digitizing on a map, requires the `geemap` module
- to install it, type `pip install geemap` into a OS terminal
- Run the next cell to see a Google map in the next cell with the hillshaded DEM and your bounding box in red
- ignore any Exceptions with with oauth2client >= 4.0.0 or google-auth, they are bogus 
- On the left, use the __Draw a Rectangle__, __Draw a Circle__ or __Draw a Polygon__ to define the shape of your print area
- Run the cell below the Map to make a GeoJSON polygon and add it to args. (If you don't want to use the shape (or didn't digitize any) and want to use the red bounding box, just skip this step.)
- Run the processing cell 

In [10]:
import geemap
center_lat = (args["trlat"] + args["bllat"]) / 2
center_lon = (args["trlon"] + args["bllon"]) / 2

Map = geemap.Map(center=(center_lat, center_lon), zoom=7) # Create an interactive map
image = ee.Image(args["DEM_name"]) # Add source DEM dataset

# make a hillshade layer and add it. 
# You can change the parameters in the next 2 lines for you needs and re-run the cell
azimuth = 315   # horizontal angle of (direction to) sun
elevation = 35  # vertical sun angle 
hs = ee.Terrain.hillshade(image, azimuth, elevation) 
vis_params = {'min': 0,'max': 255,}
Map.addLayer(hs, vis_params, 'hillshade', # Add hillshade to Map
             shown=True, opacity=0.5) 

# show bounding box
rect = ee.Geometry.Rectangle(
        args["bllon"], args["bllat"], args["trlon"], args["trlat"])
rect_feature = ee.Feature(rect)
Map.addLayer(rect_feature, {"color": 'FF0000'}, "bounding box", opacity=0.5)

# add gpx lines if used
if len(args["importedGPX"]) > 0: 
        multi_line_string = convert_to_GeoJSON(args["importedGPX"])
        mls_feature = ee.Feature(multi_line_string)
        Map.addLayer(mls_feature, {"color":'0000FF', "strokeWidth":"1"}, "GPX line", opacity=0.9)

Map # makes the interactive map show up as output of this cell
# Note: ignore exceptions with oauth2client >= 4.0.0 or google-auth

Traceback (most recent call last):
  File "C:\Users\charding\anaconda3_3.7\lib\site-packages\googleapiclient\discovery_cache\file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\charding\anaconda3_3.7\lib\site-packages\googleapiclient\discovery_cache\file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\charding\anaconda3_3.7\lib\site-packages\googleapiclient\discovery_cache\__init__.py", line 44, in autodetect
    from . import file_cache
  File "C:\Users\charding\anaconda3_3.7\lib\site-packages\googleapiclient\discovery_cache\file_ca

Map(center=[44.599634867901756, -108.11694999999997], controls=(WidgetControl(options=['position'], widget=HBo…

In [11]:
# in the map GUI above, digitze a rectangle, circle or polygon, then run this cell
# (if you screw up, just digitize a new shape and run this cell again, it will always use
# the last digitized shape ...)

# make GeoJSON polygon from (last) digitized polygon feature
polyft = Map.draw_last_feature
from geojson import Polygon
coords = polyft.getInfo()['geometry']['coordinates']
poly = Polygon(coords) # coords[0] is the digitized polygon, [1] etc. would be holes
args["polygon"] = poly # use this GeoJSON polygon in processing 
print("Using digitized polygon with", len(coords[0]), "points")

Using digitized polygon with 15 points


## Processing
- Running the cell below processes the data and creates a zip file with the model file(s) inside. This zip file will be inside the tmp folder (which is inside the same folder your notebook file is in).
- This may take some time! 
- During processing you'll see a star indicator (`In[*]`) and some log messages. (Those messages will also be in the logfile inside the zip)
- You may see some red messages with 10%, etc. - don't worry, that's normal

In [12]:

totalsize, full_zip_file_name = TouchTerrain.get_zipped_tiles(**args) # args are in a dict
print("\nDONE!\n\nCreated zip file", full_zip_file_name,  "%.2f" % totalsize, "Mb")


INFO:root:Using GeoJSON polygon for masking with 15 points
INFO:root:Log for creating 3D model tile(s) for  NED_-93.92_42.04 
 
INFO:root:DEM_name = USGS/NED 
INFO:root:trlat = 42.16927002 
INFO:root:trlon = -93.82342745999999 
INFO:root:bllat = 41.90345598 
INFO:root:bllon = -94.01463054 
INFO:root:printres = 0.4 
INFO:root:ntilesx = 1 
INFO:root:ntilesy = 1 
INFO:root:tilewidth = 200 
INFO:root:basethick = 0 
INFO:root:zscale = 1 
INFO:root:fileformat = STLb 
INFO:root:no_bottom = False 
INFO:root:unprojected = False 
INFO:root:no_normals = True 
INFO:root:
process started: 17:04:50.152405 
INFO:root:
Region (lat/lon):
   42.16927002 -93.82342745999999 (top right)
   41.90345598 -94.01463054 (bottom left) 
INFO:root:center at [-93.919029, 42.036362999999994]  UTM 15 N ,  EPSG:32615 
INFO:root:lon/lat size in degrees: [0.19120308000000819, 0.2658140400000022] 
Log for creating 3D model tile(s) for  NED_-93.92_42.04 
 
DEM_name = USGS/NED 
trlat = 42.16927002 
trlon = -93.8234274599999

In [13]:
# If you want to unzip the zip file, run this cell
# (You will need to do this before using k3d for visualization)

import os.path
from glob import glob
folder, file = os.path.splitext(full_zip_file_name) # get folder of zip file

# unzip the zipfile into the folder it's already in
import zipfile
zip_ref = zipfile.ZipFile(full_zip_file_name, 'r')
zip_ref.extractall(folder)
zip_ref.close()
print ("unzipped files from", full_zip_file_name, "into the folder", folder)
print (folder, "contains these files:")
for f in glob(folder + os.sep + "*.*"): print(" ", f)

unzipped files from tmp\myterrain.zip into the folder tmp\myterrain
tmp\myterrain contains these files:
  tmp\myterrain\logfile.txt
  tmp\myterrain\NED_-108.14_44.61.tif
  tmp\myterrain\NED_-108.14_44.61_tile_1_1.STL
  tmp\myterrain\NED_-93.92_42.04.tif
  tmp\myterrain\NED_-93.92_42.04_tile_1_1.STL


## Visualize the STL file(s)

- If you want to visualize your model(s), install k3d (`pip install k3d`) and run the cell below
- this won't work with OBJ files, as k3d can't read them in.
- binary STL should work, but ascii may not.


In [ ]:
import k3d
from glob import glob

# get all stl files in that folder
mesh_files = glob(folder + os.sep + "*.stl")
#print "in folder", folder, "using", mesh_files

plot = k3d.plot()

from random import randint
for m in mesh_files:
    col = (randint(0,255) << 16) + (randint(0,255) << 8) + randint(0,255) # random rgb color as hex
    print("adding to viewer:", m, hex(col))
    buf = open(m, 'rb').read()
    plot += k3d.stl(buf, color=col)
plot.display()